In [1]:
!pip install gdown

# 데이터 불러오기

In [70]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "./" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

Access denied with the following error:



 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X 



In [3]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

# 데이터셋 압축 해제
money_data.extractall('./data')

# 데이터 분리 및 정리
- Training set, Validation Set 을 8:2로 분리
- image별, label 별 분리

In [15]:
import shutil
from glob import glob
import os

won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = './data'

In [16]:
img_path = glob('./data/**/*.jpg', recursive=True)
json_path = glob('./data/**/*.json', recursive=True)

In [68]:
# 파일을 이동시킬 경로 지정
move_img_path = './data/images' # img 파일 경로
move_json_path = './data/labels' # json 파일 경로
data_path = './data/'# data 폴더 경로

# 화폐 단위별로 학습과 검증 비율을 8:2로 분리하기
for won in won_list:
    imgs = sorted((glob(f"{os.path.join(data_path,won,)}/*.jpg")))
    jsons = sorted((glob(f"{os.path.join(data_path,won,)}/*.json")))
    
    img_train_len = int(len(imgs)*0.8)
    json_train_len = int(len(jsons)*0.8)
    try:
        for img in imgs[:img_train_len]:
            name = img.split("\\")[-1]
            shutil.move(img, os.path.join(move_img_path, 'train', name))
        for img in imgs[img_train_len:]:
            name = img.split("\\")[-1]
            shutil.move(img, os.path.join(move_img_path, 'val', name))

        for json in jsons[:json_train_len]:
            name = json.split("\\")[-1]
            shutil.move(json, os.path.join(move_json_path, 'train', name))
        for json in jsons[json_train_len:]:
            name = json.split("\\")[-1]
            shutil.move(json, os.path.join(move_json_path, 'val', name))
    except:
        print(name)

### json에서 정보 추출
- 위치 정보 : x1,x2,y1,y2
- 박스 정보 : shape_type
- 클래스 정보 : labels

In [65]:
import json
json_path = './data/labels/'
jsons = glob(f"{json_path}/**/*.json")

In [66]:
jsons[-1]

'./data/labels\\val\\50_999_9.json'

In [67]:
set_index = {"Fifty_Thousand":7,
             'Ten_Thousand':6,
             'Five_Thousand':5,
             'Thousand':4,
             'Five_Hundred':3,
             'Hundred':2,
             'Fifty':1,
             'Ten':0,}

for j in jsons:
    with open(j, "r") as json_file:
        data = json.load(json_file)
        class_name = data['shapes'][0]['label'].replace("_back","").replace("_front","") # class 이름 설정
        class_name = set_index[class_name]
        img_w = int(data['imageWidth']) # 이미지 가로 크기
        img_h = int(data['imageHeight']) # 이미지 세로 크기
        x1,y1 = data['shapes'][0]['points'][0] # x,y 작은 값
        x2,y2 = data['shapes'][0]['points'][1] # x,y 큰 값


        center_x = (x2+x1)/2/img_w
        center_y = (y2+y1)/2/img_h
        w = (x2-x1)/img_w
        h = (y2-y1)/img_h
#         print(f"{class_name} {center_x} {center_y} {w} {h}")

        file = open(j[:-4]+'txt','w')
        file.write(f"{class_name} {center_x} {center_y} {w} {h}")
        file.close()
        

Ten_Thousand 0.48428125999999994 0.497760925 0.7111590799999998 0.24854534999999994
Ten_Thousand 0.5130514350000001 0.47922486999999997 0.71202221 0.1593197399999999
Ten_Thousand 0.52638631 0.46227763499999996 0.7555761800000002 0.26567342999999993
Ten_Thousand 0.504788468 0.46259595500000006 0.8203976039999998 0.26515863
Ten_Thousand 0.49208279 0.513755395 0.58781514 0.28261074999999997
Ten_Thousand 0.50746193 0.50870833 0.7253138600000001 0.15478134000000004
Ten_Thousand 0.51082267 0.5075197800000001 0.68727514 0.2464025000000001
Ten_Thousand 0.516085135 0.36890605 0.90495041 0.2218361
Ten_Thousand 0.525076795 0.49879642 0.7928263099999999 0.16896984
Ten_Thousand 0.534996785 0.48931986000000005 0.4936894300000001 0.30746842
Ten_Thousand 0.47725867999999994 0.522072255 0.8374706399999999 0.29188681
Ten_Thousand 0.49208096000000007 0.558176505 0.79092114 0.20534387
Ten_Thousand 0.47389303000000005 0.471280465 0.7046655399999999 0.25981853
Ten_Thousand 0.48868511 0.50611722 0.6464595 0.

Ten_Thousand 0.47368049500000003 0.5039908 0.55084873 0.197229
Ten_Thousand 0.50903323 0.48714366000000003 0.4844386000000001 0.43141168
Ten_Thousand 0.497518643 0.496176475 0.803218914 0.17555775
Ten_Thousand 0.5538165050000001 0.49183012 0.7325044700000001 0.19220064000000003
Ten_Thousand 0.50569126 0.51805325 0.79796708 0.3763636000000001
Ten_Thousand 0.49144159 0.489063795 0.70343002 0.25189209
Ten_Thousand 0.52120433 0.4291257 0.6317978399999999 0.21261673999999994
Ten_Thousand 0.49720947499999996 0.51294686 0.65850365 0.23597560000000004
Ten_Thousand 0.49212163499999995 0.47840123 0.63129223 0.21765342
Ten_Thousand 0.5022624 0.50412512 0.7681469 0.26777876000000006
Ten_Thousand 0.50675611 0.48131986 0.7503365800000001 0.19768688000000004
Ten_Thousand 0.508279065 0.5193788650000001 0.7796690699999999 0.27927987000000004
Ten_Thousand 0.512546735 0.5158247100000001 0.66912133 0.41456217999999995
Ten_Thousand 0.539273625 0.47831632999999996 0.68374609 0.15430354000000007
Ten_Thousand

Ten_Thousand 0.47753381999999994 0.6060105 0.6921045600000001 0.3448205999999999
Ten_Thousand 0.5261563968632376 0.48274179960920227 0.4713627805820106 0.1622037626461723
Ten_Thousand 0.48042748655202827 0.46776168420634917 0.6936864731040565 0.24255202841269838
Ten_Thousand 0.544108595 0.5312335154761904 0.8848542385714286 0.5737234452380953
Ten_Thousand 0.5089161633068783 0.45299997894179894 0.7706221099999999 0.28260504000000003
Ten_Thousand 0.51952998 0.445040735 0.76434364 0.19864267000000005
Ten_Thousand 0.5153458629374202 0.4837410749972633 0.8006346744170773 0.3655318985367633
Ten_Thousand 0.49955051258297256 0.49866334565656567 0.7614208603559401 0.2652995713131312
Ten_Thousand 0.5182929936281726 0.509341615 0.6538125899999999 0.3057286300000001
Ten_Thousand 0.5072357998835979 0.5112763692063492 0.8770627917671957 0.3117726584126984
Ten_Thousand 0.506323718305728 0.48390771435204355 0.6963856130013035 0.31755379144774165
Ten_Thousand 0.48107971000000005 0.430764 0.934499119999

Ten_Thousand 0.52037628 0.4873905544499815 0.5401694399999999 0.14444601
Ten_Thousand 0.500320043 0.67792985 0.7761719140000001 0.2693731
Ten_Thousand 0.5110182370820668 0.5811222982100641 0.7430525401650021 0.17176418439716315
Ten_Thousand 0.53477527 0.54276411 0.68576798 0.18552138000000007
Ten_Thousand 0.45125110229276894 0.5733644731040565 0.6434634038800705 0.22631448412698413
Ten_Thousand 0.47588402999999996 0.5539062349999999 0.57672554 0.21157992999999997
Ten_Thousand 0.5046013899999999 0.7411959300000001 0.50057548 0.3264480600000001
Ten_Thousand 0.44860469000000003 0.607961655 0.3475966200000001 0.35059236999999993
Ten_Thousand 0.567027435 0.5734921099999999 0.7171337299999999 0.18889457999999992
Ten_Thousand 0.45632373499999995 0.7238019649999999 0.65515723 0.23582827
Ten_Thousand 0.4961354420068027 0.551873058739397 0.5671008159863946 0.26180440252120596
Ten_Thousand 0.5600259651185577 0.5084325396825397 0.6534146580442877 0.16947751322751325
Ten_Thousand 0.50083936 0.48859

Ten_Thousand 0.527291605 0.46782406000000004 0.6719285100000001 0.2357876
Ten_Thousand 0.517643675 0.5744029749999999 0.8167519900000001 0.32106854999999995
Ten_Thousand 0.48907721000000004 0.49396077999999993 0.6318413 0.16531815999999996
Ten_Thousand 0.5421863650000001 0.45712898 0.6834347500000001 0.15513183999999997
Ten_Thousand 0.472647203 0.41412593500000006 0.786073994 0.17562243000000005
Ten_Thousand 0.5105620249999999 0.50631747 0.75503435 0.26290006
Ten_Thousand 0.44934581 0.5605276 0.5361155800000001 0.45460940000000005
Ten_Thousand 0.49479211 0.5509877150000001 0.60505678 0.38403083
Ten_Thousand 0.47929292 0.5668294749999999 0.65051128 0.30718499
Ten_Thousand 0.50764779 0.52682473 0.76498022 0.27513234
Ten_Thousand 0.528287025 0.514684835 0.7218728700000001 0.26122232999999995
Ten_Thousand 0.511017355 0.48935787499999994 0.7067974899999999 0.31386175
Ten_Thousand 0.5086699897227973 0.5484845891085806 0.7698120794455946 0.34511731821716124
Ten_Thousand 0.46552413350000005 0.

...

Fifty_Thousand 0.554781945 0.6465013500000001 0.6715909900000001 0.2366555
Fifty_Thousand 0.44535050750000005 0.46008984999999997 0.7580233850000001 0.21901098000000002
Fifty_Thousand 0.5290402249999999 0.44337092 0.64422141 0.16257335999999992
Fifty_Thousand 0.49885759999999996 0.5125551 0.6542646000000001 0.23410999999999996
Fifty_Thousand 0.4941606300000001 0.512086945 0.4989917400000001 0.17234501000000002
Fifty_Thousand 0.5354599775 0.49474302500000006 0.8358816450000001 0.17588635000000005
Fifty_Thousand 0.51856793 0.5213401000000001 0.74974974 0.19622800000000004
Fifty_Thousand 0.49002917 0.56717119 0.74923286 0.26198661999999995
Fifty_Thousand 0.49185453500000004 0.5757709 0.6684289300000001 0.22591479999999994
Fifty_Thousand 0.60452463 0.6573824 0.4889560600000001 0.17106753999999996
Fifty_Thousand 0.47881066 0.54820678 0.6961834800000001 0.14245515999999997
Fifty_Thousand 0.48958925499999995 0.55475797 0.80947969 0.20555225999999993
Fifty_Thousand 0.573150335 0.59857771 0.505

Fifty_Thousand 0.5143498599999999 0.714778265 0.5007410799999998 0.2486469300000001
Fifty_Thousand 0.5553537000000001 0.59823395 0.6046816 0.2097251
Fifty_Thousand 0.50675741 0.5931984299999999 0.73349438 0.24738761999999995
Fifty_Thousand 0.5000768 0.498585075 0.40049280000000004 0.32054455
Fifty_Thousand 0.5238444800000001 0.50904508 0.67703024 0.23782512000000006
Fifty_Thousand 0.514590955 0.5509067 0.5848458899999999 0.1972246
Fifty_Thousand 0.5294770750000001 0.614954185 0.74321585 0.18626822999999998
Fifty_Thousand 0.50156828 0.48300375000000006 0.7443251200000001 0.16125938000000012
Fifty_Thousand 0.47820463799999996 0.531142315 0.852513604 0.17419617000000007
Fifty_Thousand 0.48233161500000005 0.569704955 0.49091450999999997 0.16513748999999997
Fifty_Thousand 0.4040674275 0.6504520199999999 0.6726999450000001 0.3078999599999999
Fifty_Thousand 0.53504266 0.48933517000000004 0.58480748 0.35011146000000004
Fifty_Thousand 0.502265395 0.5984056249999999 0.6215730900000002 0.15636045

Five_Thousand 0.521451745 0.53259672 0.74891711 0.20465384000000011
Five_Thousand 0.51875339 0.53691969 0.6329096999999999 0.23016562
Five_Thousand 0.49085831999999996 0.6960525000000001 0.60995836 0.22802080000000008
Five_Thousand 0.49884533499999995 0.49743024 0.7315651299999999 0.26182492
Five_Thousand 0.523628355 0.49486979999999997 0.58010269 0.21202320000000008
Five_Thousand 0.5094457099999999 0.65526955 0.8311949200000002 0.3000913
Five_Thousand 0.487739 0.49305059 0.6983815999999998 0.25326922
Five_Thousand 0.4869880956469456 0.6016253022727273 0.6913884087061087 0.29415969545454557
Five_Thousand 0.476018945 0.4811605 0.5588075100000001 0.1976907999999999
Five_Thousand 0.49398125618225713 0.5397067924025274 0.6001266876354857 0.1629588151949454
Five_Thousand 0.505127645 0.500111025 0.5814217700000001 0.15889414999999998
Five_Thousand 0.49162257495590833 0.5795304232804234 0.6955467372134039 0.2538029100529101
Five_Thousand 0.5745628500000001 0.7712685249999999 0.349937440000000

Five_Hundred 0.509487865 0.52572486 0.29529892999999996 0.22090947999999994
Five_Hundred 0.5092669 0.5080619900000001 0.30179134 0.22700182
Five_Hundred 0.49640711 0.49546442500000004 0.38455418 0.17914255000000007
Five_Hundred 0.510597675 0.45799015 0.40441554999999996 0.18770530000000002
Five_Hundred 0.49918164 0.5030697700000001 0.53188464 0.25885966
Five_Hundred 0.5039995700000001 0.518411275 0.5232692600000002 0.2552492499999999
Five_Hundred 0.51856228 0.47578662 0.27275235999999997 0.14633576
Five_Hundred 0.51871395 0.419178095 0.2723186999999999 0.14707046999999998
Five_Hundred 0.511722785 0.45336297999999997 0.26882389 0.20160724
Five_Hundred 0.47332814000000006 0.48009187000000003 0.30980998000000004 0.23285586000000008
Five_Hundred 0.5142175 0.47851826999999997 0.2958366 0.16622725999999993
Five_Hundred 0.530592935 0.5139043200000001 0.36640259 0.20517090000000002
Five_Hundred 0.477827595 0.5266490200000001 0.19866492999999996 0.14880596000000004
Five_Hundred 0.500171485 0.55

Fifty 0.4897652 0.5514977400000001 0.29910839999999994 0.22245458000000012
Fifty 0.488318905 0.5269149550000001 0.36228384999999996 0.26978803
Fifty 0.42846782 0.45049 0.21781824000000005 0.16252019999999998
Fifty 0.5498256100000001 0.5948799 0.2359991 0.1740268000000001
Fifty 0.5151992449999999 0.48129341 0.22931276999999997 0.1078965
Fifty 0.507575545 0.50046077 0.21217245000000004 0.09995694000000001
Fifty 0.495974125 0.5073136800000001 0.30812955000000003 0.23122236000000007
Fifty 0.5164934 0.48484337999999993 0.3089526000000001 0.23208175999999997
Fifty 0.49738702999999995 0.51786725 0.3146427399999999 0.2323723
Fifty 0.502679625 0.51069457 0.31157155000000003 0.22981026
Fifty 0.49862480000000003 0.521072835 0.29248460000000004 0.21782732999999996
Fifty 0.5005948100000001 0.5198421 0.32776568 0.24396279999999995
Fifty 0.4729975149999999 0.5323991650000001 0.16263442999999994 0.12418827000000003
Fifty 0.46752109000000003 0.515989705 0.23384591999999996 0.1748053099999999
Fifty 0.48

# yaml 파일 만들기

In [3]:
import yaml

In [5]:
data = {
    'path':'./data'
    'train':'./data/images/train',
    'val':'./data/images/val',
    'names':{0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'},
    'nc':8
}
with open('./yamdata.yaml','w') as f:
    yaml.dump(data,f)